In [13]:
import sys, os
import torch as t
from torch import nn
from torch.autograd import Variable
import tqdm
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader

class Config(object):
    data_path = 'data/'  # 诗歌的文本文件存放路径
    pickle_path = 'D:/project/ml/test/rnn/input/tang.npz'  # 预处理好的二进制文件
    author = None  # 只学习某位作者的诗歌
    constrain = None  # 长度限制
    category = 'poet.tang'  # 类别，唐诗还是宋诗歌(poet.song)
    lr = 1e-3
    weight_decay = 1e-4
    use_gpu = False
    epoch = 1
    batch_size = 128
    maxlen = 125  # 超过这个长度的之后字被丢弃，小于这个长度的在前面补空格
    plot_every = 20  # 每20个batch 可视化一次
    # use_env = True # 是否使用visodm
    env = 'poetry'  # visdom env
    max_gen_len = 200  # 生成诗歌最长长度
    debug_file = '/tmp/debugp'
    model_path = 'D:/project/ml/data/tang_199.pth'  # 预训练模型路径
    prefix_words = '细雨鱼儿出，微风燕子斜。'  # 不是诗歌的组成部分，用来控制生成诗歌的意境
    start_words = '闲云潭影日悠悠'  # 诗歌开始
    acrostic = False  # 是否是藏头诗
    model_prefix = 'checkpoints/tang'  # 模型保存路径

opt = Config()

class PoetryModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(PoetryModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers=2)
        self.linear1 = nn.Linear(self.hidden_dim, vocab_size)

    def forward(self, input, hidden=None):
        seq_len, batch_size = input.size()
        if hidden is None:
            #  h_0 = 0.01*torch.Tensor(2, batch_size, self.hidden_dim).normal_().cuda()
            #  c_0 = 0.01*torch.Tensor(2, batch_size, self.hidden_dim).normal_().cuda()
            h_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
        else:
            h_0, c_0 = hidden
        # size: (seq_len,batch_size,embeding_dim)
        embeds = self.embeddings(input)
        # output size: (seq_len,batch_size,hidden_dim)
        output, hidden = self.lstm(embeds, (h_0, c_0))

        # size: (seq_len*batch_size,vocab_size)
        output = self.linear1(output.view(seq_len * batch_size, -1))
        return output, hidden
    
def generate(model, start_words, ix2word, word2ix, prefix_words=None):
    """
    给定几个词，根据这几个词接着生成一首完整的诗歌
    start_words：u'春江潮水连海平'
    比如start_words 为 春江潮水连海平，可以生成：

    """
    
    results = list(start_words)
    start_word_len = len(start_words)
    # 手动设置第一个词为<START>
    input = t.Tensor([word2ix['<START>']]).view(1, 1).long()
    if opt.use_gpu: input = input.cuda()
    hidden = None

    if prefix_words:
        for word in prefix_words:
            output, hidden = model(input, hidden)
            input = input.data.new([word2ix[word]]).view(1, 1)

    for i in range(opt.max_gen_len):
        output, hidden = model(input, hidden)

        if i < start_word_len:
            w = results[i]
            input = input.data.new([word2ix[w]]).view(1, 1)
        else:
            top_index = output.data[0].topk(1)[1][0].item()
            w = ix2word[top_index]
            results.append(w)
            input = input.data.new([top_index]).view(1, 1)
        if w == '<EOP>':
            del results[-1]
            break
    return results


def gen_acrostic(model, start_words, ix2word, word2ix, prefix_words=None):
    """
    生成藏头诗
    start_words : u'深度学习'
    生成：
    深木通中岳，青苔半日脂。
    度山分地险，逆浪到南巴。
    学道兵犹毒，当时燕不移。
    习根通古岸，开镜出清羸。
    """
    results = []
    start_word_len = len(start_words)
    input = (t.Tensor([word2ix['<START>']]).view(1, 1).long())
    if opt.use_gpu: input = input.cuda()
    hidden = None

    index = 0  # 用来指示已经生成了多少句藏头诗
    # 上一个词
    pre_word = '<START>'

    if prefix_words:
        for word in prefix_words:
            output, hidden = model(input, hidden)
            input = (input.data.new([word2ix[word]])).view(1, 1)

    for i in range(opt.max_gen_len):
        output, hidden = model(input, hidden)
        top_index = output.data[0].topk(1)[1][0].item()
        w = ix2word[top_index]

        if (pre_word in {u'。', u'！', '<START>'}):
            # 如果遇到句号，藏头的词送进去生成

            if index == start_word_len:
                # 如果生成的诗歌已经包含全部藏头的词，则结束
                break
            else:
                # 把藏头的词作为输入送入模型
                w = start_words[index]
                index += 1
                input = (input.data.new([word2ix[w]])).view(1, 1)
        else:
            # 否则的话，把上一次预测是词作为下一个词输入
            input = (input.data.new([word2ix[w]])).view(1, 1)
        results.append(w)
        pre_word = w
    return results

def gen(**kwargs):
    """
    提供命令行接口，用以生成相应的诗
    """

    for k, v in kwargs.items():
        setattr(opt, k, v)
    data, word2ix, ix2word = get_data(opt)
    model = PoetryModel(len(word2ix), 128, 256);
    map_location = lambda s, l: s
    state_dict = t.load(opt.model_path, map_location=map_location)
    model.load_state_dict(state_dict)

    if opt.use_gpu:
        model.cuda()

    # python2和python3 字符串兼容
    if sys.version_info.major == 3:
        if opt.start_words.isprintable():
            start_words = opt.start_words
            prefix_words = opt.prefix_words if opt.prefix_words else None
        else:
            start_words = opt.start_words.encode('ascii', 'surrogateescape').decode('utf8')
            prefix_words = opt.prefix_words.encode('ascii', 'surrogateescape').decode(
                'utf8') if opt.prefix_words else None
    else:
        start_words = opt.start_words.decode('utf8')
        prefix_words = opt.prefix_words.decode('utf8') if opt.prefix_words else None

    start_words = start_words.replace(',', u'，') \
        .replace('.', u'。') \
        .replace('?', u'？')

    gen_poetry = gen_acrostic if opt.acrostic else generate
    result = gen_poetry(model, start_words, ix2word, word2ix, prefix_words)
    print(''.join(result))

def get_data(opt):
    """
    @param opt 配置选项 Config对象
    @return word2ix: dict,每个字对应的序号，形如u'月'->100
    @return ix2word: dict,每个序号对应的字，形如'100'->u'月'
    @return data: numpy数组，每一行是一首诗对应的字的下标
    """
    if os.path.exists(opt.pickle_path):
        data = np.load(opt.pickle_path)
        data, word2ix, ix2word = data['data'], data['word2ix'].item(), data['ix2word'].item()
        return data, word2ix, ix2word

    # 如果没有处理好的二进制文件，则处理原始的json文件
    data = _parseRawData(opt.author, opt.constrain, opt.data_path, opt.category)
    words = {_word for _sentence in data for _word in _sentence}
    word2ix = {_word: _ix for _ix, _word in enumerate(words)}
    word2ix['<EOP>'] = len(word2ix)  # 终止标识符
    word2ix['<START>'] = len(word2ix)  # 起始标识符
    word2ix['</s>'] = len(word2ix)  # 空格
    ix2word = {_ix: _word for _word, _ix in list(word2ix.items())}

    # 为每首诗歌加上起始符和终止符
    for i in range(len(data)):
        data[i] = ["<START>"] + list(data[i]) + ["<EOP>"]

    # 将每首诗歌保存的内容由‘字’变成‘数’
    # 形如[春,江,花,月,夜]变成[1,2,3,4,5]
    new_data = [[word2ix[_word] for _word in _sentence]
                for _sentence in data]

    # 诗歌长度不够opt.maxlen的在前面补空格，超过的，删除末尾的
    pad_data = pad_sequences(new_data,
                             maxlen=opt.maxlen,
                             padding='pre',
                             truncating='post',
                             value=len(word2ix) - 1)

    # 保存成二进制文件
    np.savez_compressed(opt.pickle_path,
                        data=pad_data,
                        word2ix=word2ix,
                        ix2word=ix2word)
    return pad_data, word2ix, ix2word

In [6]:
data, word2ix, ix2word = get_data(opt)
data = t.from_numpy(data)

In [15]:
''.join([ix2word[x.item()] for x in data[10]])

'</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><START>夏景已难度，怀贤思方续。乔树落疎阴，微风散烦燠。伤离枉芳札，忻遂见心曲。蓝上舍已成，田家雨新足。讬邻素多欲，残帙犹见束。日夕上高斋，但望东原绿。<EOP>'

In [18]:
data.shape

torch.Size([57580, 125])

In [22]:
index = np.random.choice(range(data.shape[0]), size=1)
''.join([ix2word[x.item()] for x in data[index.item()]])

'</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><START>紫烟楼阁碧纱亭，上界诗仙独自行。奇险驱回还寂寞，云山经用始鲜明。藕绡纹缕裁来滑，镜水波涛滤得清。昏思愿因秋露洗，幸容堦下礼先生。<EOP>'

In [7]:
len(word2ix)

8293

In [58]:
dataloader = DataLoader(data,
                         batch_size=opt.batch_size,
                         shuffle=True,
                         num_workers=1)

# 模型定义
model = PoetryModel(len(word2ix), 128, 256)
optimizer = t.optim.Adam(model.parameters(), lr=opt.lr)
criterion = nn.CrossEntropyLoss()

l = []
for epoch in range(opt.epoch):
    for ii, data_ in tqdm.tqdm(enumerate(dataloader)):

        # 训练
        data_ = data_.long().transpose(1, 0).contiguous()
        optimizer.zero_grad()
        input_, target = Variable(data_[:-1, :]), Variable(data_[1:, :])
        output, _ = model(input_)
        loss = criterion(output, target.view(-1))
        loss.backward()
        optimizer.step()


        # 可视化
        if (1 + ii) % opt.plot_every == 0:

            l.append(loss.item())
            print('loss', loss.item())
            # 诗歌原文
#             poetrys = [[ix2word[_word.item()] for _word in data_[:, _iii]]
#                        for _iii in range(data_.size(1))][:16]
#             txt = '</br>'.join([''.join(poetry) for poetry in poetrys])
#             print('origin:', txt)

#             gen_poetries = []
#             # 分别以这几个字作为诗歌的第一个字，生成8首诗
#             for word in list(u'春江花月夜凉如水'):
#                 gen_poetry = ''.join(generate(model, word, ix2word, word2ix))
#                 gen_poetries.append(gen_poetry)
#             txt = '</br>'.join([''.join(poetry) for poetry in gen_poetries])
#             print('generate:', txt)





0it [00:00, ?it/s]



1it [00:04,  4.12s/it]



2it [00:07,  3.85s/it]



3it [00:11,  3.84s/it]



4it [00:15,  3.82s/it]



5it [00:19,  3.84s/it]



6it [00:24,  4.03s/it]



7it [00:28,  4.08s/it]



8it [00:32,  4.12s/it]



9it [00:38,  4.23s/it]



10it [00:42,  4.27s/it]



11it [00:47,  4.27s/it]



12it [00:51,  4.31s/it]



13it [00:56,  4.37s/it]



14it [01:01,  4.42s/it]



15it [01:06,  4.47s/it]



16it [01:11,  4.50s/it]



17it [01:16,  4.52s/it]



18it [01:21,  4.53s/it]



19it [01:26,  4.56s/it]

loss 3.632603883743286






20it [01:31,  4.57s/it]



21it [01:36,  4.57s/it]



22it [01:40,  4.57s/it]



23it [01:44,  4.56s/it]



24it [01:49,  4.56s/it]



25it [01:53,  4.56s/it]



26it [01:58,  4.56s/it]



27it [02:02,  4.55s/it]



28it [02:07,  4.55s/it]



29it [02:12,  4.55s/it]



30it [02:16,  4.55s/it]



31it [02:21,  4.58s/it]



32it [02:26,  4.59s/it]



33it [02:31,  4.58s/it]



34it [02:35,  4.58s/it]



35it [02:40,  4.58s/it]



36it [02:45,  4.58s/it]



37it [02:50,  4.60s/it]



38it [02:55,  4.61s/it]



39it [03:00,  4.62s/it]

loss 3.09531831741333






40it [03:05,  4.64s/it]



41it [03:10,  4.65s/it]



42it [03:16,  4.67s/it]



43it [03:22,  4.71s/it]



44it [03:26,  4.70s/it]



45it [03:31,  4.69s/it]



46it [03:35,  4.68s/it]



47it [03:39,  4.68s/it]



48it [03:44,  4.67s/it]



49it [03:48,  4.66s/it]



50it [03:52,  4.66s/it]



51it [03:57,  4.66s/it]



52it [04:02,  4.66s/it]



53it [04:07,  4.67s/it]



54it [04:11,  4.66s/it]



55it [04:16,  4.65s/it]



56it [04:20,  4.65s/it]



57it [04:24,  4.64s/it]



58it [04:29,  4.64s/it]



59it [04:33,  4.64s/it]

...

loss 2.854132890701294






220it [16:43,  4.56s/it]



221it [16:47,  4.56s/it]



222it [16:52,  4.56s/it]



223it [16:56,  4.56s/it]



224it [17:01,  4.56s/it]



225it [17:05,  4.56s/it]



226it [17:09,  4.56s/it]



227it [17:14,  4.56s/it]



228it [17:18,  4.55s/it]



229it [17:22,  4.55s/it]



230it [17:27,  4.55s/it]



231it [17:31,  4.55s/it]



232it [17:36,  4.55s/it]



233it [17:40,  4.55s/it]



234it [17:45,  4.55s/it]



235it [17:49,  4.55s/it]



236it [17:54,  4.55s/it]



237it [17:58,  4.55s/it]



238it [18:03,  4.55s/it]



239it [18:07,  4.55s/it]

loss 2.6666290760040283






240it [18:12,  4.55s/it]



241it [18:16,  4.55s/it]



242it [18:21,  4.55s/it]



243it [18:26,  4.55s/it]



244it [18:31,  4.55s/it]

KeyboardInterrupt: 

In [57]:
loss.item()

2.6752970218658447

In [19]:
x = data_[:,0]

In [20]:
x

tensor([8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8291, 4038, 2958, 3189, 1175, 4624, 7066, 7630, 3520,
        4710, 1161,  745, 7435, 6225, 3929, 4744, 3286, 7310, 7066, 1787, 1360,
        8010, 6787, 7914, 7435, 1719, 2808, 6016, 4782, 4414, 7066, 7854, 7377,
        6663, 3243, 1540, 7435, 8033, 2884, 4054, 3465, 8150, 7066, 3969, 5135,
        5283,   70, 7905, 7435, 8290])

In [21]:
ix2word[8292]

'</s>'

In [24]:
a = t.Tensor([1])

In [25]:
a.item()

1.0

In [26]:
poetrys = [[ix2word[_word.item()] for _word in data_[:, _iii]]
                       for _iii in range(data_.size(1))][:16]

In [28]:
txt = '</br>'.join([''.join(poetry) for poetry in poetrys])
txt

'</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><START>谷中春日暖，渐忆掇茶英。欲及清明火，能销醉客酲。松花飘鼎泛，兰气入瓯轻。饮罢闲无事，扪萝溪上行。<EOP></br><START>游人夜到汝阳间，夜色冥濛不解颜。谁家暗起寒山烧，因此明中得见山。山头山下须臾满，历险缘深无暂断。焦声散著羣树鸣，炎气傍林一川暖。是时西北多海风，吹上连天光更雄。浊烟熏月黑，高豔爇云红。初谓炼丹仙灶里，还疑铸劒神谿中。划为飞电来照物，乍作流星并上空。</br><START>少年落魄楚汉间，风尘萧瑟多苦颜。自言管葛竟谁许，长吁莫错还闭关。一朝君王垂拂拭，剖心输丹雪胸臆。忽蒙白日回景光，直上青云生羽翼。幸陪鸞辇出鸿都，身骑飞龙天马驹。王公大人借颜色，金璋紫绶来相趋。当时结交何纷纷，片言道合惟有君。待吾尽节报明主，然后相携</br></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><START>天子卹疲瘵，坤灵奉其职。年年济世功，贵贱相兼植。因产众草中，所希采者识。一枝当若神，千金亦何直。生草不生药，无以彰土德。生药不生草，无以彰奇特。国忠在臣贤，民患凭药力。灵草犹如此，贤人岂多得。<EOP></br></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><

In [66]:
gen_poetries = []
# 分别以这几个字作为诗歌的第一个字，生成8首诗
for word in list(u'春'):
    gen_poetry = ''.join(generate(model, word, ix2word, word2ix))
    gen_poetries.append(gen_poetry)
txt = '\n'.join([''.join(poetry) for poetry in gen_poetries])

In [67]:
txt

'春来不见人不知，人家女儿弄金屋。天涯相见不可论，万里千山万丈余。青荧数点奇奇士，白日葱茏生八区。东山桃李夹城东，西陵道路人不同。顾君余笑为谁子，忆昔陈家亦相望。一从遇此学为名，十年为客无遗名。君不见东西衮衮客，一年何处无人识。君今不见东方来，今日独有江州吟。君不见邺中有美酒，江上一行何处寻。使我哀心自此乐，一夜独向江南去。'

In [35]:
model

PoetryModel(
  (embeddings): Embedding(8293, 128)
  (lstm): LSTM(128, 256, num_layers=2)
  (linear1): Linear(in_features=256, out_features=8293, bias=True)
)

In [36]:
input = Variable(t.Tensor([word2ix['<START>']]).view(1, 1).long())
input

tensor([[8291]])

In [38]:
output, hidden = model(input)

In [39]:
top_index = output.data[0].topk(1)[1][0]
w = ix2word[top_index.item()]

In [40]:
w

'</s>'

In [63]:
model = PoetryModel(len(word2ix), 128, 256);
map_location = lambda s, l: s
state_dict = t.load('D:/project/ml/data/tang_199.pth', map_location=map_location)
model.load_state_dict(state_dict)

In [14]:
gen()

闲云潭影日悠悠，千里万里无人愁。人生不及春已暮，妾家独歌长安曲。西楼月，月明中，一夜东风吹晓雨。今年花落春风起，杨柳青青鸦在水。柳条繁，柳条垂，柳条丝管，杯酒满袖，歌语声尽画蛾眉。妆成蹋，望山云，复君心。妾心明月月如珠翠，绣户垂珰。陵上一行春半，月明双燕燕来。一曲弦歌，女笛歌回。愿言君王镇魏王女，传歌金马不成回。第一斗斛三五，三十六宫千万里。金龙雄劒佩金梭，玉关银汉照金绳。帐前珠佩芙蓉幕，蜡炬光辉
